In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv(override=True)

# Read from .env
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

# Assign back into os.environ (ensures consistency for libraries)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# Debug check
print("GOOGLE_API_KEY:", GOOGLE_API_KEY[:8] + "..." if GOOGLE_API_KEY else "❌ MISSING")
print("TAVILY_API_KEY:", TAVILY_API_KEY[:8] + "..." if TAVILY_API_KEY else "❌ MISSING")
print("GROQ_API_KEY:", GROQ_API_KEY[:8] + "..." if GROQ_API_KEY else "❌ MISSING")
print("LANGCHAIN_API_KEY:", LANGCHAIN_API_KEY[:8] + "..." if LANGCHAIN_API_KEY else "❌ MISSING")
print("LANGCHAIN_PROJECT:", LANGCHAIN_PROJECT if LANGCHAIN_PROJECT else "❌ MISSING")


GOOGLE_API_KEY: AIzaSyBy...
TAVILY_API_KEY: tvly-dev...
GROQ_API_KEY: gsk_Znmv...
LANGCHAIN_API_KEY: lsv2_pt_...
LANGCHAIN_PROJECT: prequist-langgraph


# Simple Ai Assistant

#### Using Google Generative AI with LangChain

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
)

query = input("Type your question (or 'exit'): ")
if query.lower() != "exit":
    response = llm.invoke(query)
    print("Response:", response.content)


d:\Project\AI\LangGraph\AI_Assistant-RAG-Tool\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Response: Hi there! How can I help you today?


#### Using Groq

In [ ]:
# from langchain_groq import ChatGroq

# llm = ChatGroq(
#     groq_api_key=os.getenv("GROQ_API_KEY"),
#     model="llama-3.1-8b-instant"
# )

# query = input("Type your question (or 'exit'): ")
# if query.lower() != "exit":
#     response = llm.invoke(query)
#     print("Response:", response.content)


Using model: llama-3.1-8b-instant
Response: I'm an artificial intelligence model known as a Large Language Model (LLM). I'm a computer program designed to understand and generate human-like text. I'm here to assist and communicate with you in a helpful and informative way.

I don't have a personal identity, emotions, or consciousness like a human being. I exist solely to provide information, answer questions, and engage in conversations based on my training data.

I'm constantly learning and improving my responses based on the interactions I have with users like you. My goal is to provide accurate, helpful, and engaging information to the best of my abilities.

So, who am I? I'm a helpful AI assistant, here to assist you with any questions or topics you'd like to discuss!


In [3]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [4]:
store={}

In [5]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [6]:
config = {"configurable": {"session_id": "firstchat"}}

In [7]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [8]:
model_with_memory.invoke(("Hi! I'm chamindu"),config=config).content

'Hi Chamindu!  How can I help you today?'

In [9]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Chamindu.'

In [10]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm chamindu", additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Chamindu!  How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--569b7adc-b26d-4538-8859-d5d63a7e796b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 14, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Chamindu.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--e038bb17-1847-4c70-b00c-12aa9221e5ad-0', usage_metadata={'input_tokens': 28, 'output_tokens': 8, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})])}

# RAG-LCEL Pre-requisite Assistant

In [11]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

### Reading the txt files from source directory

loader = DirectoryLoader('data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [15]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [16]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [17]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided text, Llama 3 is a model with at least one 8B parameter version.  Three important points are:

1. It has an 8B parameter version.
2. It was released in April 2024.
3.  It is used by at least two services (mentioned only as "Both services").


# Tools and Agents

In [18]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [19]:
api_wrapper = WikipediaAPIWrapper(top_k_results=4, doc_content_chars_max=100)

In [25]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [27]:
tool.name

'wikipedia'

In [26]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [23]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [28]:
tool.return_direct

False

In [29]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [30]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

In [ ]:
from pydantic import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Define args schema correctly
class WikiInputs(BaseModel):
    query: str = Field(..., description="The search query for Wikipedia")

# Initialize the API wrapper
api_wrapper = WikipediaAPIWrapper()

# Create the tool
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="Look up things in Wikipedia",
    args_schema=WikiInputs,  
    api_wrapper=api_wrapper,
    return_direct=True,
)


In [41]:
print(tool.name)
print(tool.description)
print(tool.args_schema)
print(tool.api_wrapper)
print(tool.return_direct)

wiki-tool
Look up things in Wikipedia
<class '__main__.WikiInputs'>
wiki_client=<module 'wikipedia' from 'd:\\Project\\AI\\LangGraph\\AI_Assistant-RAG-Tool\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'> top_k_results=3 lang='en' load_all_available_meta=False doc_content_chars_max=4000
True


In [45]:
print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more approximate nearest neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.
Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the

### youtube search tool

In [52]:
from langchain_community.tools import YouTubeSearchTool

In [53]:
tool2= YouTubeSearchTool()

In [54]:
print(tool2.name)
print(tool2.description)
print(tool2.args)
print(tool2.return_direct)

youtube_search
search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional
{'query': {'title': 'Query', 'type': 'string'}}
False


In [55]:
tool2.run("videokatha youtube chanel")

ModuleNotFoundError: No module named 'youtube_search'